In [1]:
import time, os, glob, cv2, random,os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import scale
from sklearn.metrics import roc_auc_score
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#print(tf.config.list_physical_devices('GPU'))

In [10]:
machines = ['fan','pump','slider','valve']
kinds = ['normal', 'abnormal']
labels = ["abnormal", "normal"]
img_size = 400
inputshape = (400,400,3)
rate = 0.2
rootpath = f'/mnt/Mel/Mel/slider'

In [11]:
def name_path(name):
    paths = []
    for machine in machines:
        paths.append(rootpath+f'{machine}/{name}')
    return paths

file_names = {'normal': name_path(kinds[0]),'abnormal': name_path(kinds[1])}

def RaTe(num,rate):
    t = int(num * rate)
    R = []
    while(1):
        for i in range(num):
            if t == 0:
                R.append(0)
            else:
                r = random.random()
                if r <= rate:
                    R.append(1)
                    t -= 1
                else:
                    R.append(0)
        if t == 0:
            break
        else:
            continue
    return R

In [12]:
time_start = time.time()  # time = 0

train_data=[]
test_data=[]

for label in labels:
    path = os.path.join(rootpath, label)   # create path
    class_num = labels.index(label)   # get the classification  (0 or a 1). 0=Abnormal 1=normal
    l = len(os.listdir(path))
    R = RaTe(l, rate)
    i=0
    for img in os.listdir(path):      # iterate over each image per two of them
        if R[i] == 0:
            img_arr = cv2.imread(os.path.join(path, img))[..., ::-1]  # convert BGR to RGB format
            resized_arr = cv2.resize(img_arr, (400,400))  # Reshaping images to preferred size
            train_data.append([resized_arr, class_num])
        else:
            img_arr = cv2.imread(os.path.join(path, img))[..., ::-1]  # convert BGR to RGB format
            resized_arr = cv2.resize(img_arr, (400,400))  # Reshaping images to preferred size
            test_data.append([resized_arr, class_num])
        i += 1
        
train = np.array(train_data,dtype=object)
test = np.array(test_data,dtype=object)

time_end = time.time()
minute = (time_end - time_start) // 60
second = (time_end - time_start) % 60
print('\nData Processing', minute, 'min ', second, ' sec')


Data Processing 3.0 min  51.110387563705444  sec


In [13]:
print(train[0,0].shape)
print(test[0,0].shape)
print('Train len',len(train))
print('Test len',len(test))

(400, 400, 3)
(400, 400, 3)
Train len 3297
Test len 797


In [14]:
x_train = []
y_train = []
x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)
for feature, label in test:
    x_test.append(feature)
    y_test.append(label)
# Normalize the data
x_train = np.array(x_train) 
x_test = np.array(x_test) 
print('Next')
x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)
print('Next')
x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)
print('Next')

Next
Next
Next


In [15]:
datagen = ImageDataGenerator(
    rescale=1. / 255,    
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.2, # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
datagen.fit(x_train)
print('Next')

Next


In [16]:
###model###model###model###model###model###model
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=inputshape))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 400, 400, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 200, 200, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 64)       

In [17]:
time_start = time.time()

opt = Adam(learning_rate=0.000001)
model.compile(optimizer = opt , 
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , 
              metrics = ['accuracy'])
history = model.fit(x_train,y_train,epochs = 1000 , validation_data = (x_test, y_test))

time_end = time.time()  # time = 0
minute = (time_end - time_start) // 60
second = (time_end - time_start) % 60
print('\nModel Time cost', minute, 'min ', second, ' sec')

Epoch 1/1000
104/104 [==============================] - 5s 41ms/step - loss: 14.3367 - accuracy: 0.6539 - val_loss: 12.8646 - val_accuracy: 0.2196
Epoch 2/1000
104/104 [==============================] - 3s 31ms/step - loss: 9.4729 - accuracy: 0.6679 - val_loss: 0.6250 - val_accuracy: 0.8168
Epoch 3/1000
104/104 [==============================] - 3s 32ms/step - loss: 5.9870 - accuracy: 0.6940 - val_loss: 4.1060 - val_accuracy: 0.7804
Epoch 4/1000
104/104 [==============================] - 3s 31ms/step - loss: 4.2987 - accuracy: 0.6891 - val_loss: 1.2541 - val_accuracy: 0.7817
Epoch 5/1000
104/104 [==============================] - 3s 31ms/step - loss: 2.7997 - accuracy: 0.6928 - val_loss: 0.8874 - val_accuracy: 0.7804
Epoch 6/1000
104/104 [==============================] - 3s 31ms/step - loss: 1.6794 - accuracy: 0.6985 - val_loss: 0.8614 - val_accuracy: 0.7804
Epoch 7/1000
104/104 [==============================] - 3s 32ms/step - loss: 1.2611 - accuracy: 0.7122 - val_loss: 0.7651 - val_

KeyboardInterrupt: 

In [18]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1000)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.savefig('acc2.png')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('loss2.png')
plt.show()

NameError: name 'history' is not defined

In [ ]:
acc = np.array(acc)
np.save('1000acc.npy',acc)
val_acc = np.array(val_acc)
np.save('1000val_acc.npy',val_acc)
loss = np.array(loss)
np.save('1000loss.npy',loss)
val_loss = np.array(val_loss)
np.save('1000val_loss.npy',val_loss)